In [1]:
import os
print(os.getcwd())
os.chdir("../")

/Users/pandhari/Desktop/Coffee_Shop_ChatBot/Python_Backend/Backend/experiments


In [2]:
import os
print(os.getcwd())
os.chdir("../")

/Users/pandhari/Desktop/Coffee_Shop_ChatBot/Python_Backend/Backend


In [3]:
print(os.getcwd())

/Users/pandhari/Desktop/Coffee_Shop_ChatBot/Python_Backend


In [4]:
from Backend.graph.coffee_shop_graph import build_coffee_shop_graph
build = build_coffee_shop_graph()

/opt/anaconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# from IPython.display import Image , display
# display(Image(build.get_graph(xray=True).draw_mermaid_png()))

In [5]:
from App.chatbot import get_bot_response

In [6]:
print(get_bot_response(user_input="tell me about wifi availability at shop" , user_id=15))


Starting get_user_memory
Done get_user_memory
Starting get_summary
Done get_summary
Starting get_messages
Done get_messages
Starting get_order
Started Get Order
Done get_order
Saving Order
Started Serializing Order
{'response': "Hi Pratik! I'm glad you asked about our WiFi! We do have free high-speed WiFi available at Merry's Way coffee shop. It's a great spot to relax and get some work done while enjoying your cappuccinos. \n\nIf you need any more information or if there's anything else I can assist you with, feel free to let me know!", 'state': {'user_memory': UserMemory(name='Pratik', likes=[], dislikes=[], allergies=[], last_order=None, feedback=[], location=None), 'messages': [HumanMessage(content='Tell me about the WiFi availability at the shop.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Pratik! I'm glad you asked about our WiFi! We do have free high-speed WiFi available at Merry's Way coffee shop. It's a great spot to relax and get some work done while 

In [7]:
!pip show langchain_openai

Name: langchain-openai
Version: 0.3.28
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/envs/ai/lib/python3.10/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: langchain-pinecone


In [6]:

from Backend.utils.memory_manager import get_user_memory
from Backend.utils.summary_memory import get_summary , get_messages , get_order

graph = build_coffee_shop_graph() 

In [13]:
import asyncio
import json
from rich import print

async def test_graph_stream(user_input: str, user_id: int):
    state = {
        "user_memory": None,
        "messages": [],
        "chat_summary": "",
        "user_input": "",
        "response_message": None,
        "decision": None,
        "target_agent": None,
        "order": [],
        "final_price": None,
        "memory_node": False,
    }

    state["user_input"] = user_input
    state["user_memory"] = get_user_memory(user_id)
    state["chat_summary"] = get_summary(id=user_id)
    state["messages"] = get_messages(id=user_id)
    state["order"], state["final_price"] = get_order(id=user_id)

    config = {
        "configurable": {
            "user_id": user_id
        }
    }

    print("[bold green]🚀 Starting graph stream...[/bold green]\n")

    async for event in graph.astream(state, config=config):
        if "node" in event:
            node = event["node"]
            print(f"[bold blue]🔧 Node running: {node}[/bold blue]")
        if "at" in event:
            print(f"[dim]🕒 {event['at']}[/dim]")

    print("\n✅ [bold green]Done.[/bold green]")

# Run it
await test_graph_stream("I want to order it", user_id=123)

Started Get Order


🚀 Starting graph stream...

items=[ProductItemInput(name='Latte', quantity=3)]


/opt/anaconda3/envs/ai/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Saving Order
Started Serializing Order


✅ Done.